# 1. Đăng nhập 

In [3]:
import pandas as pd
from FiinQuantX import FiinSession
import json
from datetime import datetime
# --- 1. Đăng nhập ---
username = 'DSTC_38@fiinquant.vn'
password = 'Fiinquant0606'
client = FiinSession(username=username, password=password).login()

# 2. Crawl Data from 2022 - 2025 

In [2]:
# --- Lấy danh sách mã VN30 và VNMid---
tickers_vn30 = list(client.TickerList(ticker="VN30"))
tickers_vnmid = list(client.TickerList(ticker="VNMid"))
tickers = tickers_vn30 + tickers_vnmid

data = client.Fetch_Trading_Data(
    realtime=False,
    tickers=tickers,
    fields = ['open', 'high', 'low', 'close', 'volume', 'bu','sd', 'fb', 'fs', 'fn'], 
    adjusted=True,
    by="1d",
    from_date="2019-01-01",
    to_date=datetime.today().strftime("%Y-%m-%d") 
).get_data()

Fetching data, it may take a while. Please wait...


# 3. Indicators: 

In [3]:
fi = client.FiinIndicator()

# RSI (14 ngày)
data['RSI_14'] = fi.rsi(data['close'], window=14)

# MA20, MA50, MA200
data['MA20'] = fi.sma(data['close'], window=20)
data['MA50'] = fi.sma(data['close'], window=50)
data['MA200'] = fi.sma(data['close'], window=200)

# ATR (20 ngày)
data['ATR20'] = fi.atr(data['high'], data['low'], data['close'], window=20)

# Volume trung bình 20 ngày
data['Vol_Avg20'] = fi.sma(data['volume'], window=20)

print(data.tail(30))  
data.to_csv("data/data.csv")

       ticker         timestamp      open      high       low     close  \
161391    VTP  2025-07-23 00:00  121100.0  122100.0  120400.0  121000.0   
161392    VTP  2025-07-24 00:00  121000.0  121900.0  120300.0  120400.0   
161393    VTP  2025-07-25 00:00  120500.0  123300.0  120100.0  121500.0   
161394    VTP  2025-07-28 00:00  121800.0  123000.0  121300.0  121700.0   
161395    VTP  2025-07-29 00:00  122000.0  122000.0  114000.0  114000.0   
161396    VTP  2025-07-30 00:00  112000.0  116000.0  112000.0  114000.0   
161397    VTP  2025-07-31 00:00  114200.0  115400.0  110700.0  111000.0   
161398    VTP  2025-08-01 00:00  110600.0  112700.0  107300.0  109000.0   
161399    VTP  2025-08-04 00:00  112000.0  113500.0  110600.0  113500.0   
161400    VTP  2025-08-05 00:00  114500.0  115500.0  112600.0  115000.0   
161401    VTP  2025-08-06 00:00  116600.0  117500.0  113800.0  117400.0   
161402    VTP  2025-08-07 00:00  118500.0  118600.0  115500.0  116800.0   
161403    VTP  2025-08-08

# 4. Crawl Financial Report from 2022 to 2025

In [ ]:
fa = client.FundamentalAnalysis()
all_ratios = []
for year in range(2019, datetime.now().year + 1):
    ratios = fa.get_ratios(
        tickers=tickers,
        TimeFilter="Quarterly",   
        NumberOfPeriod=4,         
        LatestYear=year,
        Consolidated=True
    )
    all_ratios.extend(ratios)
df = pd.DataFrame(all_ratios)
print(df.head(20))
print("Số dòng:", len(df))
df.to_csv("data/fundamental_ratios.csv", index=False, encoding="utf-8-sig")
print("✅ Đã lưu fundamental_ratios.csv từ 2019 đến nay")

# 5. Parsing to take out indicators: PE, PB, ROE, EPS,....

In [1]:
import pandas as pd
import ast

def extract_ratios(df: pd.DataFrame, drop_empty: bool = True) -> pd.DataFrame:
    # Chuyển cột ratios từ string sang dict, bỏ qua giá trị trống
    def safe_eval(x):
        if pd.isna(x) or str(x).strip() == "":
            return {}
        return ast.literal_eval(x)

    df['ratios'] = df['ratios'].apply(safe_eval)
    if drop_empty:
        df = df[df['ratios'].apply(lambda r: len(r) > 0)].reset_index(drop=True)
    def flatten_ratios(ratios_dict):
        flat = {}
        for category, metrics in ratios_dict.items():
            for key, value in metrics.items():
                if key == "BasicEPS":
                    flat["EPS"] = value
                elif key == "PriceToBook":
                    flat["PB"] = value
                elif key == "PriceToEarning":
                    flat["PE"] = value
                else:
                    flat[key] = value
        return flat

    ratios_expanded = df['ratios'].apply(flatten_ratios).apply(pd.Series)
    df_final = pd.concat([df.drop(columns=['ratios']), ratios_expanded], axis=1)

    return df_final

df = pd.read_csv("data/fundamental_ratios_2019_2025.csv")
df_extracted = extract_ratios(df)

output_file = "data/fundamental_ratios_extracted_2019_2025.csv"
df_extracted.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ Đã lưu file kết quả tại: {output_file}")


✅ Đã lưu file kết quả tại: data/fundamental_ratios_extracted_2019_2025.csv


# 6. Crawl VNINDEX

In [4]:
vnindex = client.Fetch_Trading_Data(
    realtime=False,
    tickers=["VNINDEX"],  
    fields=["close"],
    adjusted=True,
    by="1d",
    from_date="2019-01-01",
    to_date=datetime.today().strftime("%Y-%m-%d")
).get_data()


vnindex["daily_return"] = vnindex["close"].pct_change()
vnindex["equity_curve"] = (1 + vnindex["daily_return"].fillna(0)).cumprod()


vnindex.to_csv("data/vnindex_equity.csv", index=False, encoding="utf-8-sig")

print("✅ Saved to vnindex_equity.csv")
print(vnindex.head())
print(vnindex.tail())

Fetching data, it may take a while. Please wait...
✅ Saved to vnindex_equity.csv
    ticker         timestamp   close  daily_return  equity_curve
0  VNINDEX  2019-01-02 00:00  891.75           NaN      1.000000
1  VNINDEX  2019-01-03 00:00  878.22     -0.015172      0.984828
2  VNINDEX  2019-01-04 00:00  880.90      0.003052      0.987833
3  VNINDEX  2019-01-07 00:00  889.64      0.009922      0.997634
4  VNINDEX  2019-01-08 00:00  887.44     -0.002473      0.995167
       ticker         timestamp    close  daily_return  equity_curve
1660  VNINDEX  2025-08-27 00:00  1672.78      0.003088      1.875840
1661  VNINDEX  2025-08-28 00:00  1680.86      0.004830      1.884900
1662  VNINDEX  2025-08-29 00:00  1682.21      0.000803      1.886414
1663  VNINDEX  2025-09-03 00:00  1681.30     -0.000541      1.885394
1664  VNINDEX  2025-09-04 00:00  1696.29      0.008916      1.902204
